In [88]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [13]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [19]:
train_data = pd.read_csv('./mall.csv')

In [20]:
X = train_data.iloc[:,[3,4]].values

In [34]:
# Convert nparry to csv
print(type(X))
np.savetxt('./processed_data.csv', X, delimiter=',',fmt='%d')

<class 'numpy.ndarray'>


In [39]:
# Upload the processed csv to S3

import sagemaker
sagemaker.s3.S3Uploader.upload('processed_data.csv','s3://sampal-pi5/output', kms_key=None)



's3://sampal-pi5/output/processed_data.csv'

In [42]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sage-kmeans:latest'.format(account, region)
model = sage.estimator.Estimator(image,
                       role, 
                       train_instance_count=1, 
                       train_instance_type='ml.m5.large',
                       output_path="s3://sampal-pi5/output",
                       sagemaker_session=sess)



In [43]:
model.fit('s3://sampal-pi5/input/mall.csv')

2020-03-28 01:25:30 Starting - Starting the training job...
2020-03-28 01:25:31 Starting - Launching requested ML instances.........
2020-03-28 01:27:05 Starting - Preparing the instances for training.........
2020-03-28 01:28:58 Downloading - Downloading input data...
2020-03-28 01:29:05 Training - Downloading the training image.Starting the training.
Training complete.

2020-03-28 01:29:40 Uploading - Uploading generated training model
2020-03-28 01:29:40 Completed - Training job completed
Training seconds: 42
Billable seconds: 42


In [93]:
from sagemaker.predictor import csv_serializer
predictor = model.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

Using already existing model: sage-kmeans-2020-03-28-01-25-29-970


-------------!

In [94]:
predictor.predict(train_data.values)

b'3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n3\n4\n1\n4\n3\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n1\n2\n0\n2\n1\n2\n0\n2\n0\n2\n1\n2\n0\n2\n0\n2\n0\n2\n0\n2\n1\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n0\n2\n'

In [87]:
runtime = boto3.Session().client('sagemaker-runtime')
endpoint='sage-kmeans-2020-03-28-01-25-29-970'
import io
from io import StringIO
test_file = io.StringIO()
train_data.to_csv(test_file)
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='text/csv', Body=test_file.getvalue())
type(response)

dict

In [76]:
import json
result = json.loads(response['Body'].read().decode())
print (result)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [86]:
print(response['Body'].read().decode())

In [83]:
response_payload = json.loads(response['Body'].read().decode("utf-8"))

print ("response_payload: {}".format(response_payload))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
!curl https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/sage-kmeans-2020-03-28-01-25-29-970/invocations

In [91]:
sess.delete_endpoint(predictor.endpoint)